<a href="https://colab.research.google.com/github/ThaerAlkassab/ThaerDesktop/blob/main/TDK_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install googletrans==4.0.0-rc1
!pip install PyMultiDictionary
!pip install spacy
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [7]:
from googletrans import Translator
from ipywidgets import widgets
from PyMultiDictionary import MultiDictionary
from PyMultiDictionary import DICT_WORDNET
from PyMultiDictionary import DICT_EDUCALINGO
from IPython.display import clear_output
import requests
from IPython.display import display, Audio, HTML
import ipywidgets as widgets
import inflect

In [14]:
lang = ['Choose Language','English (en)', 'German (de)', 'French (fr)']
dictionary = MultiDictionary()
translator = Translator()

dropdown = widgets.Dropdown(options=lang, description='Select a Language:')
text_input = widgets.Text(description='Enter a Word:')

# Define variables to store language code and entered word
code = None
word = None

def extract_language_code(option):
    # Extract the two-letter code from the option
    code = option.split('(')[-1].strip(')')
    return code

def on_change(change):
    global code  # Declare code as global to modify its value within the function
    if change['type'] == 'change' and change['name'] == 'value':
        selected_lang = change['new']
        code = extract_language_code(selected_lang)
        print("Selected Language Code:", code)

def on_text_submit(text):
    global word  # Declare word as global to modify its value within the function
    word = text.value
    print("Entered Word:", word)
    pronunciations = get_pronunciations(word, language=code)
    if pronunciations:
        print("Pronunciations for", word)
        for idx, pronunciation in enumerate(pronunciations['items']):
            audio_url = pronunciation['pathmp3']
            try:
                display(HTML(f'<audio controls><source src="{audio_url}" type="audio/mpeg"></audio>'))
            except Exception as e:
                print("Error displaying audio:", e)
    else:
        print("No pronunciations found.")
    if (dictionary.meaning(code, word) == ([], '', '')):
      word = translate_and_identify_singular(word, src_lang=code)

    print(dictionary.meaning(code, word))


def get_pronunciations(word, language='en', sex='m', limit=1, group_in_languages=None):
    url = 'https://apifree.forvo.com/key/09a059a0f3086d3ab04454f0dc199531/format/json/action/word-pronunciations/word/'
    params = {
        'word': word,
        'language': language,
        'sex': sex,
        'limit': limit,
        'group-in-languages': group_in_languages
    }
    response = requests.get(url + word + '/language/' + language + '/sex/' + str(sex) + '/limit/1/')
    if response.status_code == 200:
        return response.json()
    else:
        print("Error occurred:", response.text)
        return None

def find_singular(word, language='en'):
    p = inflect.engine()
    return p.singular_noun(word) or word

def translate_and_identify_singular(word, src_lang='fr', dest_lang='en'):
  if (src_lang == 'en'):
    return find_singular(word)
  translation = translator.translate(word, src=src_lang, dest=dest_lang)
  translated_word = translation.text
  singular_word = find_singular(translated_word)
  translation = translator.translate(singular_word, src=dest_lang, dest=src_lang)
  word = translation.text
  return word

dropdown.observe(on_change)
text_input.on_submit(on_text_submit)

display(dropdown)
display(text_input)


Dropdown(description='Select a Language:', options=('Choose Language', 'English (en)', 'German (de)', 'French …

Text(value='', description='Enter a Word:')

Selected Language Code: de
Selected Language Code: fr
Entered Word: fill
Pronunciations for fill
(['Verb'], 'The definition to fill in the dictionary is to fill again something that has been emptied previously. Make something completely full, make something full. La définition de remplir dans le dictionnaire est emplir de nouveau quelque chose qui a été vidé précédemment. Faire que quelque chose soit entièrement plein, rendre plein quelque chose.', '')
Entered Word: filles
Pronunciations for filles


(['Noun'], 'The definition of girl in the dictionary is female, considered from the point of view of her descent, of her origin. Female person considered in relation to her father and / or mother. La définition de fille dans le dictionnaire est personne du sexe féminin, considérée du point de vue de son ascendance, de son origine. Personne du sexe féminin considérée par rapport à son père et/ou à sa mère.', '')
